<a href="https://colab.research.google.com/github/simonsny/image_background_removal/blob/dev_simon/bg_removal_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connecting to the Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing the necessary libraries

In [ ]:
import torch
import torch.nn

from natsort import natsorted, ns

Get the Binary Mask with U2Net

In [ ]:
import os

# clone the repository
%cd /content/drive/MyDrive/BeCode/Faktion/
if not os.path.exists('U-2-Net'):
  !git clone https://github.com/NathanUA/U-2-Net.git
%cd U-2-Net
#!python u2net_test.py

Generate the Trimap from the Binary Mask

In [ ]:
import numpy as np
import cv2

In [ ]:
def generate_trimap(mask_path,eroision_iter=6,dilate_iter=8):
    mask =  mask_path
    mask = cv2.imread(mask,0)
    mask[mask==1] = 255
    d_kernel = np.ones((3,3))
    erode  = cv2.erode(mask,d_kernel,iterations=eroision_iter)
    dilate = cv2.dilate(mask,d_kernel,iterations=dilate_iter)
    unknown1 = cv2.bitwise_xor(erode,mask)
    unknown2 = cv2.bitwise_xor(dilate,mask)
    unknowns = cv2.add(unknown1,unknown2)
    unknowns[unknowns==255]=127
    trimap = cv2.add(mask,unknowns)
    cv2.imwrite(f"/content/drive/MyDrive/trimap_mediocre/{mask_path.split('/')[-1]}",trimap)

Training the dataset with supervised_train_iter function from MODNet

In [ ]:
# Create the dataloader function


def dataloader(file_location='/content/drive/MyDrive/BeCode/Faktion/Datasets/DUTS-TR/'):
    path = file_location
    image_path = os.listdir(path + 'DUTS-TR-Image')
    gt_path = os.listdir(path + 'DUTS-TR-Ground-Truth')
    trimap_path = os.listdir(path + 'DUTS-TR-Trimap')
    
    # sort in an order
    
    """  image_path = natsorted(image_path, alg=ns.IGNORECASE)
    gt_path = natsorted(gt_path, alg=ns.IGNORECASE)
    trimap = natsorted(trimap, alg=ns.IGNORECASE)"""

    return zip(image_path, trimap_path, gt_path)

In [ ]:
# Initialize training
%cd /content/drive/MyDrive/BeCode/Faktion/MODNet
from src.models.modnet import MODNet
from src.trainer import supervised_training_iter
bs = 16         # batch size
lr = 0.01       # learn rate
epochs = 40     # total epochs

modnet = torch.nn.DataParallel(MODNet()).cuda()
optimizer = torch.optim.SGD(modnet.parameters(), lr=lr, momentum=0.9)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=int(0.25 * epochs), gamma=0.1)

/content/drive/MyDrive/BeCode/Faktion/MODNet
cannot find the pretrained mobilenetv2 backbone


NameError: ignored

In [ ]:
def supervised_training_iter(
    modnet, optimizer, image, trimap, gt_matte,
    semantic_scale=10.0, detail_scale=10.0, matte_scale=1.0):
    """ Supervised training iteration of MODNet
    This function trains MODNet for one iteration in a labeled dataset.
    Arguments:
        modnet (torch.nn.Module): instance of MODNet
        optimizer (torch.optim.Optimizer): optimizer for supervised training 
        image (torch.autograd.Variable): input RGB image
                                         its pixel values should be normalized
        trimap (torch.autograd.Variable): trimap used to calculate the losses
                                          its pixel values can be 0, 0.5, or 1
                                          (foreground=1, background=0, unknown=0.5)
        gt_matte (torch.autograd.Variable): ground truth alpha matte
                                            its pixel values are between [0, 1]
        semantic_scale (float): scale of the semantic loss
                                NOTE: please adjust according to your dataset
        detail_scale (float): scale of the detail loss
                              NOTE: please adjust according to your dataset
        matte_scale (float): scale of the matte loss
                             NOTE: please adjust according to your dataset
    
    Returns:
        semantic_loss (torch.Tensor): loss of the semantic estimation [Low-Resolution (LR) Branch]
        detail_loss (torch.Tensor): loss of the detail prediction [High-Resolution (HR) Branch]
        matte_loss (torch.Tensor): loss of the semantic-detail fusion [Fusion Branch]
    """

    global blurer

    # set the model to train mode and clear the optimizer
    modnet.train()
    optimizer.zero_grad()

    # forward the model
    pred_semantic, pred_detail, pred_matte = modnet(image, False)

    # calculate the boundary mask from the trimap
    boundaries = (trimap < 0.5) + (trimap > 0.5)

    # calculate the semantic loss
    gt_semantic = F.interpolate(gt_matte, scale_factor=1/16, mode='bilinear')
    gt_semantic = blurer(gt_semantic)
    semantic_loss = torch.mean(F.mse_loss(pred_semantic, gt_semantic))
    semantic_loss = semantic_scale * semantic_loss

    # calculate the detail loss
    pred_boundary_detail = torch.where(boundaries, trimap, pred_detail)
    gt_detail = torch.where(boundaries, trimap, gt_matte)
    detail_loss = torch.mean(F.l1_loss(pred_boundary_detail, gt_detail))
    detail_loss = detail_scale * detail_loss

    # calculate the matte loss
    pred_boundary_matte = torch.where(boundaries, trimap, pred_matte)
    matte_l1_loss = F.l1_loss(pred_matte, gt_matte) + 4.0 * F.l1_loss(pred_boundary_matte, gt_matte)
    matte_compositional_loss = F.l1_loss(image * pred_matte, image * gt_matte) \
        + 4.0 * F.l1_loss(image * pred_boundary_matte, image * gt_matte)
    matte_loss = torch.mean(matte_l1_loss + matte_compositional_loss)
    matte_loss = matte_scale * matte_loss

    # calculate the final loss, backward the loss, and update the model 
    loss = semantic_loss + detail_loss + matte_loss
    loss.backward()
    optimizer.step()

    # for test
    return semantic_loss, detail_loss, matte_loss


In [ ]:
# Training
import os

file_location='/content/drive/MyDrive/BeCode/Faktion/'

for epoch in range(0, epochs):
    for idx, (image_path, trimap_path, gt_matte_path) in enumerate(dataloader()):
        image_path = os.path.join(file_location, "images", image_path)
        trimap_path = os.path.join(file_location, "trimaps", trimap_path)
        gt_matte_path = os.path.join(file_location, "ground_truths", gt_matte_path)

        print(image_path)
        print(trimap_path)
        print(gt_matte_path)
        image = cv2.imread(image_path)
        trimap = cv2.imread(trimap_path)
        gt_matte = cv2.imread(gt_matte_path)

        semantic_loss, detail_loss, matte_loss = \
        supervised_training_iter(modnet, optimizer, image, trimap, gt_matte)
        lr_scheduler.step()

/content/drive/MyDrive/BeCode/Faktion/images/n07930864_7479.jpg
/content/drive/MyDrive/BeCode/Faktion/trimaps/n03188531_12720.png
/content/drive/MyDrive/BeCode/Faktion/ground_truths/n04285008_9860.png


NameError: ignored